In [16]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [27]:
def create_nn(batch_size=200, learning_rate=0.01, epochs=10,
              log_interval=10):

    train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
            batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])),
            batch_size=batch_size, shuffle=True)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(28 * 28, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 10)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)

    net = Net()

    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.NLLLoss()

    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.data))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 28 * 28)
        net_out = net(data)
        test_loss += criterion(net_out, target).data
        pred = net_out.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [28]:
if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()

<ipython-input-27-138194497991>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.308460
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.193995
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.960422
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.388081
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.852852
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.650394
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.521520
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.366918
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.433707
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.431550
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.429556
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.331912
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.409538
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.404296
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.334511
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.263415
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.244100
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.200110
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.409208
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.397409
Train Epoch

<ipython-input-27-138194497991>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.0004, Accuracy: 9773/10000 (98%)

